### QLearning From Scratch Using Numpy
We explore a agent to find the cheese problem. 

In [1]:
import numpy as np
import random

# Q Learning Pseudo Code

1. Initialze the Q-values (Q(s,a)) arbitrarily for all state-action pairs.
2. For life or untill learning is stopped
3.  Choose an action a in the current world state (s)
4.Take the action (a) and observe the outcome state(s') and reward(r)
5. Update `Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s', a') - Q(s,a)]`

## Environment As:
### States:
```
 _________________________
|Agent|1 cheese|3 cheese |
|________________________|
|2Cheese|Poison|BigCheese|
|________________________|
```

0-> Blank

1 -> 1 Cheese

2 -> Poison

3 -> 2 Cheeses

4 -> 3 Cheeses

5 -> Big Cheese


### Actions:
0 -> Left

1 -> Right

2 -> Up

3 -> Down


If the agent take 1 cheese-> +1 reward, 2 cheese-> +2 reward, 3 cheese => +3 reward Big Cheese-> +10 reward, Poison-> -10 reward.


In [2]:
# Create The Environment
states = np.array([[0, 1, 4],
                  [3, 2, 5]]) # define states as shown in above scenerio
actions = np.array([0, 1, 2, 3]) # Actions: left, right, up, down

In [33]:
class Cheese_Agent:
    
    def __intit__(self):
        self.state = 0
        self.new_state = 0
        self.reward = 0
        self.done = False
    def step(self,action):
        
        if action == 0: # left
            if self.state == 1: # from 1 to left, agent goes to blank position, so reward =0
                self.reward = 0
                self.new_state = 0
            elif self.state == 4: # from 4 to left, agent eat 1 cheese
                self.reward = 1
                self.new_state = 1
            elif self.state == 2: # From 2 to left, agent eat 2 cheeses
                self.reward = 2
                self.new_state = 3
            elif self.state == 5: # From 5 to left, agent eat poison; so penalize by reward=-10
                self.reward = -10
                self.new_state = 2
            else: 
                reward = 0 
                # Also can be when agent try to go up in  0, 1, 4 then, out of scope which can be reward as -1 but now is 0
            
            
        if action == 1: # right
            if self.state == 0: # From 0 to right, Agent take 1 cheese
                self.reward = 1
                self.new_state = 1
            elif self.state == 1: # From 1 to right, Agent take 3 cheese
                self.reward = 3
                self.new_state = 4
            elif self.state == 3: # From 3 to right is poison
                self.reward = -10  # highly penalize the agent on taking poison
                self.new_state = 2
            elif self.state == 2:
                self.reward = 10 # The goal is to eat big cheese, so episode complete; done=True
                self.new_state = 5
                self.done = True
            else: 
                self.reward = -1
            
        if action ==2: # UP
            if self.state == 3: # From 3 to UP, agent goes in blank position i.e no reward
                self.reward = 0
                self.new_state = 0
            elif self.state == 2: # from 2 to UP, agent eat 1 cheese
                self.reward = 1
                self.new_state = 1
            elif self.state == 5: # from 5 to UP, agent eat 3 cheeses
                self.reward = 3
                self.new_state = 4
            else: 
                self.reward = -1
            
        if action ==3: # down
            if self.state == 0: # From 0 to down, Agent take 2 cheese
                self.reward = 2
                self.new_state = 3
            elif self.state == 1: # From 1 to down, Agent take poison, so reward = -10
                self.reward = -10
                self.new_state = 2
            elif self.state == 4: 
                self.reward = 10  # The goal is to eat big cheese, so episode complete; done = True
                self.new_state = 5
                self.done = True
            else: 
                self.reward = -1
                
        self.state = new_state # update state
        return self.new_state, self.reward, self.done
    
    def reset(self):
        self.state = 0  # initial position
        self.new_state = 0
        self.done = False
        return self.state

In [4]:
# Define States and actions
state_size = states.shape[0]* states.shape[1]
print("State_size: ",state_size)

action_size = len(actions)
print("Action_size:",action_size)

State_size:  6
Action_size: 4


## Initialize The Q Table

In [5]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Step3: Create the Hyperparameters

In [48]:
total_episodes = 10000                            # Total number of episodes
total_test_episodes = 500                       # Total number of test episodes
max_steps = 150                                # Total number of steps taken in an episode

learning_rate = 0.8
gamma = 0.8                                   # Discount Factor

# Exploration Parameters
epsilon = 1.0                                   # Exploration Rate
max_epsilon = 1.0                               # Exploration probability at start
min_epsilon = 0.01                              # Minimum exploration probability
decay_rate  = 0.01                              # Decay rate for exponential exploration probability


### Step 4: Q Learning Algorithm

1. Initialze the Q-values (Q(s,a)) arbitrarily for all state-action pairs.
2. For life or untill learning is stopped
3.  Choose an action a in the current world state (s)
4. Take the action (a) and observe the outcome state(s') and reward(r)
5. Update `Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s', a') - Q(s,a)]`

In [49]:
env = Cheese_Agent()

# 2 for life or untill learning is stopped
for episode in range(total_episodes):
    # Reset the Environment
    step = 0
    state = env.reset()
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            
        # Else doing a random choice --> exploration
        else:
            action = random.randint(0, action_size -1)
            
        # 4. Take the action (a) and observe the outcome state (s') and reward(r)
        new_state, reward, done = env.step(action)
        
        # 5. Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s', a') - Q(s,a)]
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state,:]) - qtable[state, action])
        
        # Our new state is state
        state = new_state
        
        # If Done: Finish Episode
        if done == True:
            break
    
    # Reduce epsilon ( as we need less and less exploration on agent goes on learning)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
print(epsilon)

0.01


In [50]:
print(qtable)

[[ 7.17543359  6.04214151  3.55252359  9.89434086]
 [ 4.13356399  9.11913858  4.11859418 -2.40040081]
 [-9.60688462 -1.60224407  6.15740631 -1.67712855]
 [ 0.07940621 -3.26048824  9.11439943  1.01788812]
 [ 7.73522292  7.21147501  2.89087827 -2.52781262]
 [ 0.          0.          0.          0.        ]]


# Step 5: Use our Q-Table to eat Cheese By Agent

In [51]:
env.reset()
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step =0
    total_rewards = 0
    if episode % 10 == 0:
        print("******************************************")
        print("EPISODE", episode)

    for step in range(max_steps):
        
        # Take the action(index) that have the maximum expected future reward  given that state
        action = np.argmax(qtable[state, :])
        new_state, reward, done = env.step(action)
        total_rewards += reward
        if done:
            rewards.append(total_rewards)
            print("Score: ", total_rewards)
            break
        state = new_state

print("Score over time: "+ str(sum(rewards)/ total_test_episodes))

******************************************
EPISODE 0
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
******************************************
EPISODE 10
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
******************************************
EPISODE 20
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
******************************************
EPISODE 30
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
******************************************
EPISODE 40
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
******************************************
EPISODE 50
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
Score:  21
*****************